# Train model and do test

In [52]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from Hyperparameter_optimization import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from proj1_helpers import *

## Load the training set
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y_train, tX_train, ids = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)

# convert y from -1/1 to 0,1
y_train = (y_train+1)/2

## Load the test set

# Load test set
DATA_TRAIN_PATH = '../data/test.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)

# Train a model and see accuracy

In [103]:
data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)
#lambda_ = [1e-2,1e-3,1e-4,1e-5]
lambda_ = [1e-4]
#lambda_ = np.linspace(1e-3,1e-5,4)
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
optimal_lambdas = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)),tx,tx**2,tx**3,tx**4,tx**5, tx**6,tx**7,tx**8,tx**9,tx**11,1/(abs(tx)+1e-10),np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=10, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'ridge_regression')
                                                                                   #model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))
    optimal_lambdas.append(optimal_lambda_)

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.5f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0.001,  gamma: 0.01 
 
 Model ridge_regression 
 Loss: 0.06845308622720489 
 Accuracy: 0.8099606992817454
────────────────────────────────────────
lambda : 0.000945,  gamma: 0.01 
 
 Model ridge_regression 
 Loss: 0.06844933827073421 
 Accuracy: 0.8099200433663098
────────────────────────────────────────
lambda : 0.0008900000000000001,  gamma: 0.01 
 
 Model ridge_regression 
 Loss: 0.06844565462037186 
 Accuracy: 0.8099335953381217
────────────────────────────────────────
lambda : 0.000835,  gamma: 0.01 
 
 Model ridge_regression 
 Loss: 0.06844202545693502 
 Accuracy: 0.8100420111126168
────────────────────────────────────────
lambda : 0.00078,  gamma: 0.01 
 
 Model ridge_regression 
 Loss: 0.06843847600374693 
 Accuracy: 0.8100826670280525
────────────────────────────────────────
lambda : 0.000725,  gamma: 0.01 

In [104]:
optimal_lambdas

[6.499999999999995e-05,
 1e-05,
 0.0008900000000000001,
 0.000835,
 0.00078,
 0.000615]

# Train model and do prediction on test set

In [102]:
from preprocess_data import *

data_pro  = data_preprocess_train(tX_train,y_train,outlier_method = 'IQR',n_out = 5)

# Train
lambda_ = [6.5e-05,
 1e-05,
 0.0009,
 0.000835,
 0.00078,
 0.000615]
loss_ = [] 
w_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9,tx**11,1/(abs(tx)+1e-10), np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    loss_temp, w_temp = ridge_regression(y, tx,lambda_[idx_data])
    w_.append(w_temp)
#print('\n * Overall Accuracy = %.4f'%accuracy_all)

## Do predicition on test set
data_test  = data_preprocess_predict(tX_test,outlier_method = 'IQR',n_out = 5)
print('Training finished, start do prediction and write to prediction.csv')
# Using trained model to do prediction
label_ = []
data_size = sum([len(data_test[:,1][i]) for i in range(6)])
y_predict = np.zeros(data_size)
for idx_data,data in enumerate(data_test):
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9,tx**11,1/(abs(tx)+1e-10), np.sin(tx)))
    label_.append(predict_labels(w_[idx_data], tx,model='ridge_regression'))
    y_predict[data[1]] = predict_labels(w_[idx_data], tx,model='ridge_regression')

create_csv_submission(range(350000,350000+data_size),(y_predict-0.5)*2,'prediction.csv')

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 3               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 4               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 5               │
└──────────────────────────────────────┘
Training finished, start do prediction and write to prediction.csv


# Others

In [191]:
# Train
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
loss_ = [] 
w_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9, np.sin(tx)))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    loss_temp, w_temp = least_squares(y, tx)
    w_.append(w_temp)
#print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 3               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 4               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 5               │
└──────────────────────────────────────┘


In [192]:
# Using trained model to do prediction
acc_ = []
for idx_data,data in enumerate(data_pro):
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9, np.sin(tx)))
    y = data[1]
    acc_.append(compute_accuracy(tx, y, w_[idx_data], model='least_squares'))
print(acc_)            

[0.8055834123865022, 0.9479385981701949, 0.7956617415906948, 0.918804549061095, 0.8279355198637578, 0.9166854820501242]


In [194]:
y_predict = np.zeros(sum([len(data_pro[:,2][i]) for i in range(6)]))
for idx_data,data in enumerate(data_pro):
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9, np.sin(tx)))
    #label_.append(predict_labels(w_[idx_data], tx,model='least_squares'))
    y_predict[data[2]] = predict_labels(w_[idx_data], tx,model='least_squares')


In [198]:
# Load test set
DATA_TRAIN_PATH = '../data/test.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
feature_names = load_headers(DATA_TRAIN_PATH)


In [199]:
data_test  = data_preprocess_predict(tX,outlier_method = 'IQR',n_out = 3)

In [200]:
# Using trained model to do prediction
acc_ = []
label_ = []
data_size = sum([len(data_test[:,1][i]) for i in range(6)])
y_predict = np.zeros(data_size)
for idx_data,data in enumerate(data_test):
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2,tx**3,tx**4,tx**5,tx**6,tx**8,tx**9, np.sin(tx)))
    label_.append(predict_labels(w_[idx_data], tx,model='least_squares'))
    y_predict[data[1]] = predict_labels(w_[idx_data], tx,model='least_squares')


In [201]:
create_csv_submission(range(350000,350000+data_size),(y_predict-0.5)*2,'prediction.csv')

# Train and predict

In [260]:
tX.shape

(568238, 30)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 3               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 4               │
└──────────────────────────────────────┘
┌──────────────────────────────────────┐
│               Group: 5               │
└──────────────────────────────────────┘


## Without bias

In [113]:
from Hyperparameter_optimization import *

In [114]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.13134087391240726 
 Accuracy: 0.6929717569252453
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.6929717569252453 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.026579363200754248 
 Accuracy: 0.9403139356814703
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9403139356814703 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└──────────────

## With bias

## With bias and second order 

In [139]:
lambda_ = [0]
gamma_ = [1e-2]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((np.ones((tx.shape[0],1)), tx,tx**2))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=4, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 100, 
                                                                                   model = 'least_squares')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.0728864008015079 
 Accuracy: 0.7952377080284058
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.7952377080284058 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 1               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 0.01 
 
 Model least_squares 
 Loss: 0.02309335642638366 
 Accuracy: 0.9424961715160797
════════════════════════════════════════
*The optimal hyperparameters*:
 Model: least_squares 
 Optimal accuracy: 0.9424961715160797 
 Optimal lambda_: 0 
 Optimal gamma: 0.01 
 
┌──────────────────────────────────────┐
│               Group: 2               │
└────────────────

### Try higher orders and non-linear term

In [168]:
lambda_ = [0]
gamma_ = [1e-5]
accuracy_ = []
group_size_ = []
for idx_data,data in enumerate(data_pro):
    print('┌'+'─' * 38+'┐')
    print('│               Group: %i               │'%idx_data)
    print('└'+'─' * 38+'┘')
    tx = data[0]
    tx = np.hstack((tx**0 ,tx))
    y = data[1]
    initial_w_ = np.ones(tx.shape[1])
    optimal_accuracy, optimal_lambda_, optimal_gamma = Hyperparameter_optimization( y, tx,k_fold=10, 
                                                                                   lambdas=lambda_ , 
                                                                                   gammas = gamma_, 
                                                                                   initial_w = initial_w_, 
                                                                                   max_iters = 1000, 
                                                                                   model = 'logistic_regression')
    accuracy_.append(optimal_accuracy)
    group_size_.append(len(data[2]))

accuracy_all = np.dot(accuracy_,group_size_)/np.sum(group_size_)
print('\n * Overall Accuracy = %.4f'%accuracy_all)

┌──────────────────────────────────────┐
│               Group: 0               │
└──────────────────────────────────────┘
────────────────────────────────────────
lambda : 0,  gamma: 1e-05 
 
Current iteration=0, loss=nan
Current iteration=100, loss=31476.345236940593
Current iteration=200, loss=31417.107922883464
Current iteration=300, loss=31405.56055855738
Current iteration=400, loss=31403.046397704355
Current iteration=500, loss=31402.487016534564
Current iteration=600, loss=31402.361488007817
Current iteration=700, loss=31402.333181678572
Current iteration=800, loss=31402.32677903084
Current iteration=900, loss=31402.32532780975
Current iteration=0, loss=31454.457524724592
Current iteration=100, loss=31452.63855977228
Current iteration=200, loss=31452.488950897867
Current iteration=300, loss=31452.46009258678
Current iteration=400, loss=31452.454141875187
Current iteration=500, loss=31452.452824808177
Current iteration=600, loss=31452.4525124258
Current iteration=700, loss=31452.